# Infinite elements

The infinite element basis functions consist of generalized Laguerre functions (cf. [Section 6.3](dissertation_wess.pdf#section.6.3) and [Definition A.10](dissertation_wess.pdf#thm.A.10)).
These functions can be evaluated using the formula [A.11(iv)](dissertation_wess.pdf#Item.54) (note that here the arguments are scaled by a factor $2$) which can be implemented in python as follows.

In [1]:
# %load -r 1-13 infinite_elements.py
from numpy import array,zeros,exp,shape,atleast_1d,eye,diag,ones

def gen_laguerre_func(n,m,x):
    x = atleast_1d(x)
    res = zeros((n+1,len(x))) 
    if x[0] is complex:
        res+=0j

    res[0,:]=exp(-x/4)
    res[1,:]=exp(-x/4)*(m+1-x)
    for i in range(1,n):
        res[i+1,:]=1/(i+1)*((2*i+m+1-x)*res[i,:]-(i+m)*res[i-1,:])
    return res*exp(-x/4)

## Assembling the infinite element matrices

The infinite element matrices can be assembled using either explicit formulas or numerical integration. 

### Using explicit formulas

A simple way to utilize explicit formulas is to represent the basis functions with index $-1$ in the orthogonal basis with index $0$ and compute the according matrices as it is done in the following.

In [2]:
# %load -r 15-32 infinite_elements.py
def ie_matrices(N):
    Tp=-eye(N+1)
    Tp[:-1,1:]-=eye(N+1-1)
    Tm=eye(N+1)
    Tm[:-1,1:]-=eye(N+1-1)

    Diffop=-1/2*(-diag(range(1,2*N+1+1,2))
                +diag(range(1,N+1),1)+diag(range(1,N+1),-1))

    ie_mass = 1/2*Tm.T@Tm
    ie_laplace = 1/2*Tp.T@Tp
    ie_drift = 1/2*Tm.T@Tp
    ie_mass_x = 1/2*Tm.T@Diffop@Tm
    ie_mass_xx = 1/2*Tm.T@Diffop@Diffop@Tm
    ie_laplace_x = 1/2*Tp.T@Diffop@Tp
    ie_laplace_xx = 1/2*Tp.T@Diffop@Diffop@Tp
    ie_drift_x = 1/2*Tm.T@Diffop@Tp
    return ie_mass,ie_laplace,ie_drift,ie_mass_x, ie_mass_xx, ie_laplace_x, ie_laplace_xx,ie_drift_x

This gives exactly the matrices from [Section 7.4.1](dissertation_wess.pdf#subsection.7.4.1)

In [3]:
M,S,D,M_x,_,_,_,_ = ie_matrices(5)
print('mass:')
print(M)
print('stiffness:')
print(S)
print('drift:')
print(D)

mass:
[[ 0.5 -0.5  0.   0.   0.   0. ]
 [-0.5  1.  -0.5  0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.   0.  -0.5  1.  -0.5]
 [ 0.   0.   0.   0.  -0.5  1. ]]
stiffness:
[[0.5 0.5 0.  0.  0.  0. ]
 [0.5 1.  0.5 0.  0.  0. ]
 [0.  0.5 1.  0.5 0.  0. ]
 [0.  0.  0.5 1.  0.5 0. ]
 [0.  0.  0.  0.5 1.  0.5]
 [0.  0.  0.  0.  0.5 1. ]]
drift:
[[-0.5 -0.5  0.   0.   0.   0. ]
 [ 0.5  0.  -0.5  0.   0.   0. ]
 [ 0.   0.5  0.  -0.5  0.   0. ]
 [ 0.   0.   0.5  0.  -0.5  0. ]
 [ 0.   0.   0.   0.5  0.  -0.5]
 [ 0.   0.   0.   0.   0.5  0. ]]


### Using numerical integration

An alternative way, which also enables us to deal with inhomogeneous exterior domains is to use numerical integration (cf. [Section 7.4.2](dissertation_wess.pdf#subsection.7.4.2)). The quadrature rules form Theorems [7.14](dissertation_wess.pdf#thm.7.14) and [7.15](dissertation_wess.pdf#thm.7.15) are implemented as follows.

In [4]:
# %load -r 34-42 infinite_elements.py
def mod_laguerre_quad(n):
    from numpy.linalg import eig
    mat=diag([(2*i+1) for i in range(n+1)])
    mat[1:,:-1]-=diag(range(1,n+1))
    mat[:-1,1:]-=diag(range(1,n+1))
    points,v = eig(mat)
    
    l=gen_laguerre_func(n,0,points)[-1,:]
    return array(points),array(points/l/l/(n+1)/(n+1))

We use this quadrature to compute the mass matrix for a given coefficient function.

In [5]:
# %load -r 44- infinite_elements.py
def ie_mass_quad(coef,N,M):
    points,weights = mod_laguerre_quad(M)
    vals = gen_laguerre_func(N,-1,points)
    coefpoints = coef(points/2)
    mat= zeros((N+1,N+1))
    for i in range(N+1):
        for j in range(N+1):
            mat[i,j]=1/2*(vals[i,:]*coefpoints*vals[j,:])@weights
    return mat


In [6]:
print(ie_mass_quad(lambda x: ones(shape(x)),5,10))

[[ 5.00000000e-01 -5.00000000e-01 -6.26554619e-15 -5.96096699e-15
  -3.66766326e-15 -1.83483400e-15]
 [-5.00000000e-01  1.00000000e+00 -5.00000000e-01  8.64698002e-15
   2.12120978e-15 -2.73666755e-15]
 [-6.26554619e-15 -5.00000000e-01  1.00000000e+00 -5.00000000e-01
  -2.35779403e-16  2.88535802e-15]
 [-5.96096699e-15  8.64698002e-15 -5.00000000e-01  1.00000000e+00
  -5.00000000e-01  1.39070782e-15]
 [-3.66766326e-15  2.12120978e-15 -2.35779403e-16 -5.00000000e-01
   1.00000000e+00 -5.00000000e-01]
 [-1.83483400e-15 -2.73666755e-15  2.88535802e-15  1.39070782e-15
  -5.00000000e-01  1.00000000e+00]]


In [7]:
print(ie_mass_quad(lambda x: x,5,10))
print(M_x)

[[ 2.50000000e-01 -5.00000000e-01  2.50000000e-01 -4.31004591e-15
  -1.05973753e-15  1.36746641e-15]
 [-5.00000000e-01  1.50000000e+00 -1.50000000e+00  5.00000000e-01
   2.30178975e-15 -5.68647459e-15]
 [ 2.50000000e-01 -1.50000000e+00  3.00000000e+00 -2.50000000e+00
   7.50000000e-01  5.13833564e-15]
 [-4.32393703e-15  5.00000000e-01 -2.50000000e+00  4.50000000e+00
  -3.50000000e+00  1.00000000e+00]
 [-1.05626808e-15  2.31566753e-15  7.50000000e-01 -3.50000000e+00
   6.00000000e+00 -4.50000000e+00]
 [ 1.37429688e-15 -5.68647459e-15  5.13833564e-15  1.00000000e+00
  -4.50000000e+00  7.50000000e+00]]
[[ 0.25 -0.5   0.25  0.    0.    0.  ]
 [-0.5   1.5  -1.5   0.5   0.    0.  ]
 [ 0.25 -1.5   3.   -2.5   0.75  0.  ]
 [ 0.    0.5  -2.5   4.5  -3.5   1.  ]
 [ 0.    0.    0.75 -3.5   6.   -4.5 ]
 [ 0.    0.    0.    1.   -4.5   7.5 ]]
